# More on classes

We begin with review and design considerations.

In [1]:
class Widget: 
    
    def __init__(self, age, weight): 
        self.age = age
        self.weight = weight
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.age!r}, {self.weight!r})'
    
    def __eq__(self, other): 
        if not isinstance(other, type(self)):
            return NotImplemented
        # return self.age == other.age and self.weight == other.weight
        return self.__dict__ == other.__dict__

In [2]:
Widget('10 years', '10 pounds')

Widget('10 years', '10 pounds')

In [3]:
Widget(10.0, 1.0) == Widget(10, 1)

True

In [4]:
Widget(10.0, 1.0) == Widget(10, 2)

False

In [5]:
class Employee: 
    
    def __init__(self, age, weight): 
        self.age = age
        self.weight = weight
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.age!r}, {self.weight!r})'
    
    def __eq__(self, other): 
        if not isinstance(other, type(self)):
            return NotImplemented
        # return self.age == other.age and self.weight == other.weight
        return self.__dict__ == other.__dict__

In [6]:
Employee(23.0, 200.0) == Employee(23, 200)

True

In [7]:
Employee(23.0, 200.0) == Employee(20, 200)

False

In [8]:
w = Widget(70, 180)

In [9]:
w == 42

False

In [10]:
e = Employee(70, 180)

In [11]:
w == e

False

In [12]:
class ExplodingWidget(Widget):
    def explode(self):
        print(f'{self!r} has exploded!')

In [13]:
ew = ExplodingWidget(70, 180)

In [14]:
ew.explode()

ExplodingWidget(70, 180) has exploded!


In [15]:
ew.explode()

ExplodingWidget(70, 180) has exploded!


In [16]:
w == ew

True

In [17]:
w.__dict__ == e.__dict__

True

## Weird derived classes (semi-review)

In [18]:
class CallableStr(str): 
    def __call__(self): 
        return self

In [19]:
f = CallableStr(312)

In [20]:
f

'312'

In [21]:
f()

'312'

In [22]:
f is f()

True

In [23]:
from decorators import joining

In [24]:
joining(f)

<function __main__.joining.<locals>.decorator.<locals>.wrapper()>

In [25]:
joining('s')

<function decorators.joining.<locals>.decorator(func)>

## Effect of defining special methods (review)

Defining special dunder methods ("magic" methods) as attributes of a class (which is most often done with a `def` statement in the class body) customizes the behavior of *instances* of the class. They should not be defined on the instances themselves (which does not work, and constitutes using a dunder name in a manner other than explicitly documented).

In [32]:
class HasRepr: 
    def __repr__(self): 
        return f'{type(self).__name__}()'

In [43]:
hr = HasRepr()

In [44]:
hr.__repr__()

'HasRepr()'

In [45]:
hr

HasRepr()

In [46]:
repr(hr)

'HasRepr()'

In [34]:
class NoRepr: 
    pass

In [40]:
nr = NoRepr()
nr.__repr__ = lambda self: f'{type(self).__name__}()'

In [42]:
nr.__repr__(nr)

'NoRepr()'

In [47]:
NoRepr.__repr__ = lambda self: f'{type(self).__name__}()'

In [48]:
nr

NoRepr()

In [49]:
nr.__repr__()

TypeError: <lambda>() missing 1 required positional argument: 'self'

In [50]:
nr2 = NoRepr()

In [51]:
nr2.__repr__()

'NoRepr()'

In [52]:
class ReallyNoRepr: 
    pass

In [54]:
rnr = ReallyNoRepr()
rnr.__repr__ = lambda: f'{type(rnr).__name__}()'

In [55]:
rnr.__repr__()

'ReallyNoRepr()'

In [56]:
rnr

## Review of `super`

In [57]:
del CallableStr  # Just to make clear that we're not reusing the old one.

In [94]:
class CallableStr(str): 
    def __repr__(self): 
        return f'{type(self).__name__}({super().__repr__()})'
    
    def __call__(self): 
        return self

In [95]:
f = CallableStr(312)

In [96]:
f

CallableStr('312')

In [97]:
CallableStr('312')

CallableStr('312')

In [86]:
CallableStr("a'b")

CallableStr("a'b")

In [87]:
CallableStr('a\'b')

CallableStr("a'b")

In [98]:
kent = super()

RuntimeError: super(): no arguments

In [99]:
class CallableInt(int): 
    def __repr__(self): 
        return f'{type(self).__name__}({super().__repr__()})'
    
    def __call__(self): 
        return self

In [100]:
g = CallableInt('312')

In [101]:
g

CallableInt(312)

In [102]:
CallableInt(312)

CallableInt(312)

## Conferring that style of `repr` via mixin class

In [106]:
class _MixinRepr:
    def __repr__(self): 
        return f'{type(self).__name__}({super().__repr__()})'

In [107]:
class CallableStrA(_MixinRepr, str): 
    def __call__(self): 
        return self

In [108]:
a = CallableStrA('a')

In [109]:
a

CallableStrA('a')

In [110]:
class CallableIntA(_MixinRepr, int): 
    def __call__(self): 
        return self

In [111]:
one = CallableIntA('1')

In [112]:
one

CallableIntA(1)

In [113]:
class BrokenCallableStrA(str, _MixinRepr):  # Lists bases in wrong order.
    def __call__(self): 
        return self

In [114]:
BrokenCallableStrA('a')

'a'

## Important sidebar: Method resolution order

In [115]:
CallableStrA.__mro__

(__main__.CallableStrA, __main__._MixinRepr, str, object)

In [116]:
BrokenCallableStrA.__mro__

(__main__.BrokenCallableStrA, str, __main__._MixinRepr, object)

## Named tuples and data classes: Motivation

In [27]:
def summarize(values):
    """
    Compute min, max, and arithmetic, geometric, and harmonic mean.

    values is an arbitrary nonempty iterable of strictly positive real numbers.
    If values is empty, or any value is nonpositive, ValueError is raised. The
    caller is responsible for ensuring each object in values represents a real
    number. The five computed results are returned as [FIXME: decide how].

    All computations are done in a single pass: values is iterated just once.
    Time complexity is O(len(values)). Space complexity is O(1). These assume a
    number takes O(1) space and arithmetic operations take O(1) time, which is
    often an approximation in Python, but it is guaranteed when using floats.
    """
    # FIXME: Implement this.

In [28]:
# TODO: Make the rest of this section.